In [7]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # this is to silence tf warnings

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_probability as tfp
from tensorflow.keras.layers import Input, Dense, Dropout
# from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
from scipy.signal import savgol_filter
# from tensorflow.keras.constraints import non_neg

In [8]:
# add these lines if you use a tf-gpu 
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

##### Declare constants

In [4]:
rng = np.random.default_rng(seed=0)
TRAIN_SPLIT_FRACTION = 0.9
VAL_SPLIT_FRACTION = 0.7
BATCH_SIZE = 128
LEARNING_RATE = 1e-04
EPOCHS = 2_000
LAMBDA = 1e-02
PATH_SAVE = '<model save path>'

##### Create custom losses

In [5]:
class RPD(tf.keras.metrics.Metric):

    def __init__(self, name="rpd", **kwargs):

        super().__init__(name=name, **kwargs)
        self.std = self.add_weight(
            name="std", initializer="zeros"
            )
        self.rmse = self.add_weight(
            name="rmse", initializer="zeros"
            )
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32"
            )

    
    def update_state(self, y_true, y_pred, sample_weight=None):

        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(num_samples)
        y_mean = tf.reduce_sum((y_true)/tf.cast(self.total_samples, tf.float32))
        std = tf.sqrt(tf.reduce_sum(tf.square(y_true - y_mean))/tf.cast(self.total_samples, tf.float32))
        self.std.assign_add(std)
        rmse = tf.sqrt(tf.reduce_sum(tf.square(y_true - y_pred))/tf.cast(self.total_samples, tf.float32))
        self.rmse.assign_add(rmse)


    def result(self):
        return  self.std/self.rmse


    def reset_state(self):        
        self.std.assign(0.)
        self.rmse.assign(0.)
        self.total_samples.assign(0)


# class RPIQR(tf.keras.metrics.Metric):

#     def __init__(self, name="rpiqr", **kwargs):

#         super().__init__(name=name, **kwargs)
#         self.rmse = self.add_weight(
#             name="rmse", initializer="zeros"
#             )
#         self.total_samples = self.add_weight(
#             name="total_samples", initializer="zeros", dtype="int32"
#             )
#         self.iqr = self.add_weight(
#             name="iqr", initializer="zeros", dtype="float32"
#             )

    
#     def update_state(self, y_true, y_pred, sample_weight=None):

#         num_samples = tf.shape(y_pred)[0]
#         self.total_samples.assign_add(num_samples)
#         percentiles = tfp.stats.percentile(y_true, q=[25., 75.])
#         self.iqr.assign_add(percentiles[1]-percentiles[0])
#         rmse = tf.sqrt(tf.reduce_sum(tf.square(y_true - y_pred))/tf.cast(self.total_samples, tf.float32))
#         self.rmse.assign_add(rmse)


#     def result(self):
#         return  self.iqr/self.rmse


#     def reset_state(self):        
#         self.iqr.assign(0.)
#         self.rmse.assign(0.)
#         self.total_samples.assign(0)

##### Load csv

In [6]:
caco3 = pd.read_csv('<path to data>')

##### Convert to numpy array

In [7]:
caco3_array = caco3.to_numpy(copy=True)

##### Shuffle 

In [8]:
rng.shuffle(caco3_array)

##### Calculate validation split

In [9]:
# VAL_SPLIT = int(VAL_SPLIT_FRACTION * caco3_array.shape[0])

##### Smooth

In [9]:
caco3_array[:, :-1] = savgol_filter(caco3_array[:, :-1], 13, polyorder = 2, deriv=2)

##### Uncomment cell below if you want to normalize

In [13]:
# scaler = Normalizer()
# scaler.fit(caco3_array[:VAL_SPLIT])
# caco3_array = scaler.transform(caco3_array)

##### Calculate train split

In [10]:
TRAIN_SPLIT = int(TRAIN_SPLIT_FRACTION * caco3_array.shape[0])

##### Split set to train and test

In [11]:
# train = caco3_array[:TRAIN_SPLIT, :-1]
# calc = caco3_array[:TRAIN_SPLIT, -1]
train = caco3_array[:, :-1]
calc = caco3_array[:, -1]

##### Declare your callbacks

In [12]:
# this callback is to stop training if the model does not improve
callback_01 = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=1e-03,
    patience=75,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=False
)

# this callback is to save model mid-training
callback_02 = tf.keras.callbacks.ModelCheckpoint(
    PATH_SAVE,
    monitor="val_loss",
    mode = 'min',
    save_best_only=True,
)

# this callback is to reduce learning rate if the gradient reaches a plateau
callback_03 = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=10,
    verbose=0,
    mode="min",
    min_delta=10e-4
)

# define the function that schedules how the learning rate changes
# this one reduces the lr by a factor of 10 every 1_500 epochs
def scheduler(epoch, lr):
    if not epoch%1_500:
        lr *= tf.math.pow(10.0, -1)
    return lr 

# def scheduler(epoch, lr):
#     # LR = 0.003
#     i = epoch//1_500
#     lr /= 10**i
#     return lr

# then pass the scheduler function to the callback
callback_04 = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [24]:
1e-6 * tf.math.pow(10.0, -1)

<tf.Tensor: shape=(), dtype=float32, numpy=1e-07>

##### Declare your regularizers

In [13]:
L1_reg = tf.keras.regularizers.L1(l1=LAMBDA)
L2_reg = tf.keras.regularizers.L2(l2=LAMBDA)

##### Model architecture

In [16]:
inputs = Input(shape=(train.shape[1]))
# inputs = tf.keras.layers.Input(shape=(inputs.shape[1], 1))
dense_01 = Dense(
    units=500,
    activation='relu',
    # kernel_regularizer=L1_reg,
    # activity_regularizer=L2_reg,
)(inputs)
# dropout_01 = Dropout(0.3)(dense_01)
dense_02 = Dense(
    units=200,
    activation='relu',
    # kernel_regularizer=L1_reg,
    # activity_regularizer=L2_reg,
)(dense_01)
dense_03 = Dense(
    units=50,
    activation='relu',
    kernel_regularizer=L1_reg,
    activity_regularizer=L2_reg,
    )(dense_02)
# dense_04 = Dense(
#     units=32,
#     activation='relu',
#     kernel_regularizer=tf.keras.regularizers.L1(0.01),
#     activity_regularizer=tf.keras.regularizers.L2(0.01)
#     )(dense_03)
# dropout_02 = Dropout(0.3)(dense_01)
# dense_03 = Dense(128, activation='relu')(dropout_02)
outputs = Dense(
    units=1,
    # activation='exponential',
    # kernel_constraint=non_neg(),
    # bias_constraint=non_neg(),    
)(dense_03)

##### Create model

In [17]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

##### Compile

In [18]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=LEARNING_RATE,
        # clipnorm=1,
    ),
    loss='mean_squared_error',
    metrics = [tfa.metrics.RSquare(), RPD()],
)

In [20]:
# loaded_model = tf.keras.models.load_model('<model load path>', custom_objects={'RPD': RPD})

##### Fit

In [2]:
history = model.fit(
    x=train,
    y= calc,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=VAL_SPLIT_FRACTION,
    callbacks=[callback_02, callback_04],  # mind the lr reducer callback!
    verbose=1
)